# import dependecies

In [25]:
import cv2
import os
import numpy as np
import uuid
from pathlib import Path
from sklearn.model_selection import train_test_split
import json

# DataSet Generation

In [10]:
def generate_dataset(name):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            unique_filename = f"{name}_{str(uuid.uuid4())}.jpg"
            file_name_path = os.path.join("data", unique_filename)
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
        
        if cv2.waitKey(1) == 13 or img_id == 200: # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

# Example usage
generate_dataset("Mahesh_Semwal")

# Save Data from LFW to Data and CROP

In [17]:
def crop_faces_and_save(source_dir, target_dir):
    # Initialize face cascade
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    # Ensure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    # Process each image in the dataset
    for person_dir in Path(source_dir).iterdir():
        if person_dir.is_dir():
            person_name = person_dir.name
            for image_path in person_dir.iterdir():
                if image_path.is_file() and image_path.suffix in [".jpg", ".jpeg", ".png"]:
                    img = cv2.imread(str(image_path))
                    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    
                    # Detect faces
                    faces = face_classifier.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=5)
                    
                    # Crop and save each detected face
                    for i, (x, y, w, h) in enumerate(faces):
                        face = gray_img[y:y+h, x:x+w]
                        unique_filename = f"{person_name}_{i}_{str(uuid.uuid4())}.jpg"
                        file_name_path = os.path.join(target_dir, unique_filename)
                        cv2.imwrite(file_name_path, face)

    print("Face cropping and saving completed for all images in 'LFW' directory.")

In [18]:
# Example usage:
source_dir = "lfw"
target_dir = "data"
crop_faces_and_save(source_dir, target_dir)

Face cropping and saving completed for all images in 'LFW' directory.


# Data Augmentation

In [26]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Function to perform data augmentation and save augmented images
def augment_and_save(image_path, target_dir):
    img = cv2.imread(image_path)
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((1,) + img.shape)
    
    i = 0
    for batch in datagen.flow(img, batch_size=1, save_to_dir=target_dir, save_prefix='aug', save_format='jpg'):
        i += 1
        if i > 4:
            break


# transefer learning Model

In [ ]:
# Function to train a transfer learning model
def train_transfer_learning_model(data_dir):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    # Example: Load and augment your dataset
    # train_generator = train_datagen.flow_from_directory(...)

    model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator), validation_data=val_generator, validation_steps=len(val_generator))

    model.save('transfer_learning_model.h5')


# one shot Learning

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Function for one-shot learning using OpenCV
def one_shot_learning(reference_img_path, query_img_path):
    reference_img = cv2.imread(reference_img_path, cv2.IMREAD_GRAYSCALE)
    query_img = cv2.imread(query_img_path, cv2.IMREAD_GRAYSCALE)

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces_ref = face_cascade.detectMultiScale(reference_img, scaleFactor=1.3, minNeighbors=5)
    faces_query = face_cascade.detectMultiScale(query_img, scaleFactor=1.3, minNeighbors=5)

    # Assuming single face detection from each image for simplicity
    if len(faces_ref) == 1 and len(faces_query) == 1:
        (x_ref, y_ref, w_ref, h_ref) = faces_ref[0]
        (x_query, y_query, w_query, h_query) = faces_query[0]

        # Crop faces
        face_ref = reference_img[y_ref:y_ref+h_ref, x_ref:x_ref+w_ref]
        face_query = query_img[y_query:y_query+h_query, x_query:x_query+w_query]

        # Resize to a fixed size for model compatibility
        face_ref = cv2.resize(face_ref, (224, 224))
        face_query = cv2.resize(face_query, (224, 224))

        # Preprocess images for model prediction
        face_ref = np.expand_dims(face_ref, axis=0) / 255.0
        face_query = np.expand_dims(face_query, axis=0) / 255.0

        # Load your transfer learning model
        model = load_model('transfer_learning_model.h5')

        # Extract features using the model
        features_ref = model.predict(face_ref)
        features_query = model.predict(face_query)

        # Compute similarity score (e.g., cosine similarity)
        similarity_score = np.dot(features_ref, features_query.T)[0, 0]

        # Determine if faces are the same based on threshold
        if similarity_score > 0.5:
            return True
        else:
            return False
    else:
        print("Error: Could not detect exactly one face in either reference or query image.")
        return None


# Real-Time Face Recognition

In [22]:
# Function to draw boundaries around detected faces and recognize them
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
         
        # Perform face recognition
        id_, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
         
        # Display recognized name if confidence is high
        if confidence > 70:
            name = reverse_label_map.get(id_, "UNKNOWN")
            cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
     
    return img



In [23]:
import cv2
import json
from tensorflow.keras.models import load_model

# Load the label map
with open("label_map.json", "r") as f:
    label_map = json.load(f)

# Reverse the label map to get names from labels
reverse_label_map = {int(v): k for k, v in label_map.items()}

# Initialize the face cascade and the LBPH face recognizer
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read('classifier.xml')

# Capture video from the webcam and perform real-time face recognition
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply the face detection and recognition
    frame = draw_boundary(frame, face_cascade, scaleFactor=1.1, minNeighbors=5, color=(255, 0, 0), clf=clf)

    # Display the result
    cv2.imshow("Face Detection", frame)

    # Exit on 'Enter' key press
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()
